# Captura de Registos Bibliográficos da Biblioteca Nacional de Portugal

## Aplicação de Web Scraping em Open Data

Os ficheiros fonte foram obtidos no repositório nacional OpenData, disponibilizados pela Biblioteca NacionaL de Portugal, em formato MarcXchange XML, em https://opendata.bnportugal.gov.pt/downloads.htm

Apliquei a livraria de web scraping BeautifulSoup, versão 4, em Python, seguindo as regras de classificação UniMarc.

As regras UNIMARC seguem a versão 3, do Catálogo Bibliográfico Abreviado, de 2008, publicado pela Biblioteca Nacional de Portugal, diponível em https://www.ifla.org/files/assets/uca/publications/unimarc-holdings-format-pt.pdf

Os resultados são guardados em ficheiros JSON e CSV.


Carlos Catalão Alves

Lisboa, Janeiro 2021

Copyright MIT

### Importação de livrarias para web scraping

In [1]:
import requests
from bs4 import BeautifulSoup 

In [475]:
# Uso Pandas só para a gravação em JSON/CSV
import pandas as pd

In [729]:
# Lista de títulos dos ficheiros XML
fnames = ['bibliographics_1_to_55654',
    'bibliographics_55655_to_118584',
    'bibliographics_118585_to_204006',
    'bibliographics_204007_to_283314',
    'bibliographics_283315_to_367718',
    'bibliographics_367719_to_470125',
    'bibliographics_470126_to_559734',
    'bibliographics_559737_to_693394',
    'bibliographics_693441_to_761849',
    'bibliographics_761850_to_897443',
    'bibliographics_897444_to_1016009',
    'bibliographics_1016010_to_1073638',
    'bibliographics_1073639_to_1192807',
    'bibliographics_1192808_to_1258277',
    'bibliographics_1258279_to_1373167',
    'bibliographics_1373168_to_1698672',
    'bibliographics_1698674_to_1753584',
    'bibliographics_1753585_to_1807726',
    'bibliographics_1807727_to_1864741',
    'bibliographics_1864742_to_1923995',
    'bibliographics_1923996_to_1984865',
    'bibliographics_1984867_to_2050311',
    'bibliographics_2050312_to_2058585',
]

### Funções usadas na captura de dados

In [710]:
def get_capa_http(bs):
    
    capa_http = ''
    try:
        tmp = str(bs.find("mx:subfield", attrs={"code":'u'}).string)
        tmp_lst = tmp.split('?')
        capa_http = tmp_lst[0]
        
        img = tmp_lst[1]
        x = img.find('img=')
        tmp = img[x:]
        tmp_lst = tmp.split('&')
        img = tmp_lst[0]
        capa_http = capa_http + '?' + img
    except:
        pass
        
    return capa_http

In [711]:
def get_bnp_notas(bs):
    
    bnp_notas = ''
    try:
        bnp_notas = str(bs.find("mx:subfield", attrs={"code":'a'}).string)
    except:
        pass
        
    return bnp_notas

In [712]:
def get_coleccao(bs):
    
    coleccao = ''
    try:
        coleccao = str(bs.find("mx:subfield", attrs={"code":'a'}).string)
    except:
        pass
        
    return coleccao

In [713]:
def get_autor_principal_sobrenome(bs):
    
    autor_principal_sobrenome = ''
    try:
        tmp = str(bs.find("mx:subfield", attrs={"code":'a'}).string)
        autor_principal_sobrenome = tmp.strip('')
        autor_principal_sobrenome = autor_principal_sobrenome.strip(',')
    except:
        pass

    return autor_principal_sobrenome

In [714]:
def get_autor_principal_nome(bs):
    
    autor_principal_nome = ''
    try:
        autor_principal_nome = str(bs.find("mx:subfield", attrs={"code":'b'}).string)
    except:
        pass

    return autor_principal_nome

In [715]:
def get_isbn(bs):
    
    isbn = ''
    try:
        isbn = str(bs.find("mx:subfield", attrs={"code":'a'}).string)
    except:
        pass

    return isbn

In [716]:
def get_cdu(bs):
    
    cdu = ''
    try:
        cdu = str(bs.find("mx:subfield", attrs={"code":'a'}).string)
    except:
        pass

    return cdu

In [717]:
def get_editora_local(editora):
    
    editora_local = ''
    
    try:
        editora_local = str(editora.find("mx:subfield", attrs={"code":'a'}).string)
    except:
        pass
    
    return editora_local

In [718]:
def get_editora_nome(editora):
    
    editora_nome = ''
    try: 
        tmp = str(editora.find("mx:subfield", attrs={"code":'c'}).string)
        editora_nome = tmp.strip(', ')
    except:
        pass
    
    return editora_nome

In [719]:
def get_editora_ano(editora):
    
    editora_ano = ''
    try:
        editora_ano = str(editora.find("mx:subfield", attrs={"code":'d'}).string)
    except:
        pass
    
    return editora_ano

In [721]:
def get_autor_secundario(bs):
    
    autor_secundario = ''
    try:
        tmp = str(bs.find("mx:subfield", attrs={"code":'a'}).string)
        autor_secundario_sobrenome = tmp.strip()
        autor_secundario_sobrenome = autor_secundario_sobrenome.strip(',')
        autor_secundario_nome = str(bs.find("mx:subfield", attrs={"code":'b'}).string)
        autor_secundario = autor_secundario_nome + ' ' + autor_secundario_sobrenome 
    except:
        pass

    return autor_secundario.replace(',','')

In [722]:
def get_titulo_original(bs):
    
    titulo_original = ''
    try:
        titulo_original = str(bs.find("mx:subfield", attrs={"code":'a'}).string)
    except:
        pass
    
    return titulo_original

In [723]:
def get_responsavel(bs):
    
    responsavel = ''
    try:
        responsavel = str(bs.find("mx:subfield", attrs={"code":'f'}).string)
    except:
        pass
    
    return responsavel

In [724]:
def get_ilustracao(bs):
    
    ilustracao = ''
    try:
        tmp = str(bs.find("mx:subfield", attrs={"code":'g'}).string)
        if tmp.find('il') >= 0:
            ilustracao = tmp
    except:
        pass
    
    return ilustracao

In [725]:
def get_traducao(bs):
    
    traducao = ''
    try:
        tmp = str(bs.find("mx:subfield", attrs={"code":'g'}).string)
        if tmp.find('trad') >= 0:
            traducao = tmp
    except:
        pass
    
    return traducao

In [726]:
def get_edicao(bs):
    
    edicao = ''
    try:
        edicao = str(bs.find("mx:subfield", attrs={"code":'a'}).string)
    except:
        pass
    
    return edicao

In [727]:
def get_descricao_fisica(bs, code):
    
    descricao_fisica = ''
    try:
        descricao_fisica = str(bs.find("mx:subfield", attrs={"code": code}).string)
    except:
        pass
    
    return descricao_fisica

### Leitura dos ficheiros XML e gravação de resultados em JSON e CSV

In [ ]:
# Ciclo principal de execução de todos os ficheiros XML 

for ficheiro in ficheiros:
    
    reg_list = []
    artigos = []
    
    fn = ficheiro + '.xml'
    soup = BeautifulSoup(open(file), features='lxml')
    artigos = soup.findAll('mx:record')
    
    # Executa o ciclo de scraping
    reg_list = get_dados_bnp(artigos)
    
    # Cria a datframe dos eresultados
    df = pd.DataFrame(reg_list)
    
    # Grava em CSV
    df.to_csv(fname +'.csv')
    
    # Grava em JSON
    df.to_csv(fname +'.csv')   

In [ ]:
# Ciclo de scraping (executado para cada um dos ficheiros XML)
def get_dados_bnp(artigos):

    for a in artigos:
    
        reg = {'registo_bnp':'','link_bnp': '',
            'responsavel': '', 'ilustracao': '', 'traducao': '',
            'edicao_no': '', 'desc_fisica': '',
            'editora_local':'', 'editora_ano':'', 'editora_nome':'',
            'tit_original':'',
            'autor_principal_nome':'', 'autor_principal_sobrenome':'',
            'autores_secundarios': '',
            'coleccao':'', 'coleccao_no':'',
            'notas_bnp': '',
            'isbn':'',
            'cdu':'',
            'capa':'',
        }

        #  link para BNP
        bs = a.find('mx:controlfield', attrs={'tag': '003'}) # link BNP
        try:
            reg['link_bnp'] = bs.text
        except:
            reg['link_bnp'] = ''

        # Bloco 200: Titulo e menção de responsabilidade
        bs = a.find('mx:datafield', attrs={'tag': '200'})
        reg['responsavel'] = get_responsavel(bs)
        reg['ilustracao'] = get_ilustracao(bs)
        reg['traducao'] = get_traducao(bs)

        # Publicação
        bs = a.find('mx:datafield', attrs={'tag': '210'}) # publicação
        reg['editora_local'] = get_editora_local(bs)
        reg['editora_ano'] = get_editora_ano(bs)
        reg['editora_nome'] = get_editora_nome(bs)

        #  Número de edição
        bs = a.find('mx:datafield', attrs={'tag': '205'}) # edicao 
        reg['edicao_no'] = get_edicao(bs)


        # Descricao fisica
        bs = a.find('mx:datafield', attrs={'tag': '215'}) # desc fisica
        codes = ['a','c','d']
        reg['desc_fisica'] = ''
        for code in codes:
            item = get_descricao_fisica(bs, code)
            reg['desc_fisica'] = reg['desc_fisica'] + ' ' + item
            reg['desc_fisica'] = reg['desc_fisica'].strip()


        #  Título original
        bs = a.find('mx:datafield', attrs={'tag': '304'}) # tit. orig.
        reg['tit_original'] = get_titulo_original(bs)

        # Autores secundarios
        bs = None
        tag = 701
        bs = a.find('mx:datafield', attrs={'tag':'701'}) # autor secundário 1
        reg['autores_secundarios'] = get_autor_secundario(bs)

        bs = a.find('mx:datafield', attrs={'tag':'702'}) # autor secundário 2
        autor_sec2 = get_autor_secundario(bs)
        if len(autor_sec2) > 0:
            reg['autores_secundarios'] = reg['autores_secundarios'] + ', ' + autor_sec2
        reg['autores_secundarios'] = reg['autores_secundarios'].strip(', ')


        # Bloco de Responsabilidade
        bs = None
        tag = 700
        while bs == None:
            bs = a.find('mx:datafield', attrs={'tag': str(tag)}) # autor principal
            tag+=1
        reg['autor_principal_sobrenome'] = get_autor_principal_sobrenome(bs)
        reg['autor_principal_nome'] = get_autor_principal_nome(bs)

        # Colecção
        bs = a.find('mx:datafield', attrs={'tag': '225'}) # coleccao
        reg['coleccao'] = get_coleccao(bs)  
        if len(reg['coleccao']):
            item = bs.find("mx:subfield", attrs={"code":'v'}) # número 
            if item:
                reg['coleccao_no'] = item.string


        # ISBN
        bs = a.find('mx:datafield', attrs={'tag': '010'}) # ISBN
        reg['isbn'] = get_isbn(bs)

        # CDU
        bs = a.find('mx:datafield', attrs={'tag': '675'}) # CDU
        reg['cdu'] = get_cdu(bs)

        # Link para imagem da capa
        bs = a.find('mx:datafield', attrs={'tag': '856'}) # capa
        reg['capa'] = get_capa_http(bs)

        reg['registo_bnp'] = str(a.find('mx:controlfield', attrs={'tag': '001'}).text)       

        print(reg)
        reg_list.append(reg)
        
    return(reg_list)


Carlos Catalão Alves

Janeiro 2021